# Generate ROMs from the Object Files in Tempest Source Disks

In [150]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:60% !important; }</style>"))

## Get the Source

Get the source files from the 'historicalsource' github repository if we don't already have it.

In [198]:
!rm -rf tempest
!git clone https://github.com/historicalsource/tempest

Cloning into 'tempest'...
remote: Enumerating objects: 45, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 45 (delta 8), reused 45 (delta 8), pack-reused 0 (from 0)
Receiving objects: 100% (45/45), 174.01 KiB | 1.89 MiB/s, done.
Resolving deltas: 100% (8/8), done.


## An Expensive Bug

There is more than one version of Tempest because some time after the game's release in October 1981 players discovered a juicy little bug that hit arcade owners where it hurts (in the pocketbook). If you managed to reach a score of 170,000 or more there was a roughly one-in-eight chance that the next coin you popped in the cabinet would give you a whopping forty credits instead of just one:

<img src="images/service.png" width=60%>

In [19]:
!diff ../src/ALDISP.MAC ../src/ALDIS2.MAC

81c81

---

339c339
< 
---

2849c2849
< ;INPUT:ACC=LEVEL #-1
---

3284d3283

\ No newline at end of file


```asm
ZATVG2::
	LDA SECUVY      ; ARE WE DISPLAYING ONE OF THE TITLE SCREENS?
	IFNE            ; IF WE ARE THEN:
	LDY I,27        ; FOR ALL 39 BYTES IN THE VECTOR DISPLAYING THE ATARI COPYRIGHT
	LDA I,0E        ; STARTING FROM OUR HARD-CODE LITERAL OF '0E'
	SEC            
	BEGIN
	SBC NY,SECUVG   ; SUBTRACT EACH BYTE TO CALCULATE A FINAL CHECKSUM VALUE.
	DEY
	MIEND
	TAY
	IFNE            ; IF THE RESULT IS ZERO, THE CHECKSUM PASSES OTHERWISE:
	EOR I,0E5       ; CHECK IT AGAINST THE CHECKSUM FOR ANOTHER SCREEN
	ENDIF           
	IFNE            ; IF THAT PASSES WE'RE DONE OTHERWISE:
	EOR I,029       ; CHECK IT AGAINST THE CHECKSUM FOR ANOTHER SCREEN
    ;!! In Revision 1, the above line had an incorrect value of 02A so the check failed when it shouldn't have.
	ENDIF           
	STA QT3         ; STORE THE RESULT OF OUR CHECK IN QT3.
	ENDIF
	ENDIF
```

```
ZQVAVG::
    LDA QT3     ; CHECK THAT BOTH QT3 AND
	ORA QT6     ; QT6 ARE ZERO.
	IFNE        ; IF THEY ARE NOT:
	LDA I,17    ; CHECK IF THE PLAYER' SCORE IS GREATER THAN 170,000
	CMP LSCORH
	IFCC        ; IF IT IS:
	LDX LSCORL  ; LOAD WHATEVER IS IN THE LSCORL BYTE
	INC X,0     ; AND USE THAT AS A REFERENCE TO INCREMENT ONE OF OUR 'ZERO-PAGE' BYTES.
	ENDIF       ; IN THE HOPE OF CAUSING SOME HAVOC.
	ENDIF
```
For example if `LSCORL` is `0x06` then it will increment by one whatever we store at address `$06`. 

The bytes that are stored at these low addresses are given in `ALCOMN.MAC`. Here are the bytes that live in addresses `$00` to `$08`:
```
QSTATE:	 .BLKB 1			;CONTAINS CODE FOR STATE ROUTINE (INDEX INTO ROUTAD)
QDSTATE: .BLKB 1			;DISPLAY STATE
QNXTSTA: .BLKB 1			;NEXT STATE CODE TO EXECUTE AFTER PAUSE
QFRAME:	 .BLKB 1			;FRAME COUNTER (WRAPS AT FF)
QTMPAUS: .BLKB 1			;PAUSE TIMER (IN SECOND UNITS)
QSTATUS: .BLKB 1			;STATUS FLAGS
$$CRDT:  .BLKB 1			;# OF CREDITS
$INTCT:  .BLKB 1			;INTERRUPT COUNT
$COINA:  .BLKB 1			;COIN MECHS
```

Incrementing once is bad enough, but since this increment would happen many times (and keep happening) the value in `$$CRDT` will eventually 
exceed 40. When this happens, a bit of logic in `ALSCO2.MAC` will kick in:

ALSCO2.MAC
```
ZATC4E:
	LDA $$CRDT
	CMP I,28
	IFCS			;MAXIMIZE # CREDITS TO 40.
	LDA I,28
```

In `ALSCO2.MAC` 

```
GENPLA:	
    JSR DPLRNO		;PLAYER X
	JMP INFO
```
is replaced with:
```
GENPLA:	
    JSR DPLRNO		;PLAYER X
	JMP HACKER
```
So instead of jumping to the top of the `INFO` routine we jump about half-way down, skipping a lot of work, including loading whatever value is
in `$$CRDT`:

```
INFO:
	LDA I,1
	STA VGSIZE
	JSR VGSCA1
	LDY I,LETCOL		;STANDARD LETTER COLOR
	JSR NWCOLO
	LDA QSTATUS
	IFPL			;ATTRACT?
	LDX I,MGAMOV		;YES. "GAME OVER"
	LDA QFRAME
	AND I,20
	IFEQ
	LDX I,MINSER		;FLASH INSERT COINS
	LDA $$CRDT
	IFNE
	BIT TCMFLG
	IFPL			;2 GAME MINIMUM?
	LDX I,MPRESS		;NO. PRESS START
	ENDIF
	ENDIF
	ENDIF
	JSR MSGS
	JSR VGCNTR
	LDA VGMSGA		;BLANK OUT LEVEL
	STA SCLEVEL
	STA SCLEVEL+2
HACKER:	
    JSR DSPCRD		;DISPLAY CREDITS & ATARI
	ENDIF
```

## But, what are the differences?

In [4]:
!diff -y -W 150 tempest/ALDISP.MAC tempest/ALDIS2.MAC |grep '  |	'

										
										
										
grep: (standard input): binary file matches


In [11]:
!diff ../src/ALHARD.MAC ../src/ALHAR2.MAC

22,23c22,23


---


28c28

---



In [14]:
!diff -b2 -a2 ../src/ALSCOR.MAC ../src/ALSCO2.MAC

74c74

---

315c315

---



In [15]:
!diff -b2 -a2 ../src/ALTEST.MAC ../src/ALTES2.MAC

21c21

---

246,247c246,247


---


265c265

---

289a290,291


291a294

293a297

402c406

---

441a446,448



497c504

---

564,633d570






































































641d577

643c579,580

---


645,646d581


648,649c583,585


---



652c588,597

---










674c619,627

---









677,685c630,642









---













690c647,705

---



























































726c741

---

733d747

754c768,775

---








757,784c778,784




























---







788,791c788,790




---



801c800,825

---


























829c853

---

832,834d855



842c863

---

852d872

899a920,929










